In [1]:
%%javascript

IPython.keyboard_manager.command_shortcuts.add_shortcut('Ctrl-k', {
    help : 'move up selected cells',
    help_index : 'jupyter-notebook:move-selection-up',
    handler : function (event) {
        IPython.notebook.move_selection_up();
        return false;
    }}
);

IPython.keyboard_manager.command_shortcuts.add_shortcut('Ctrl-j', {
    help : 'move down selected cells',
    help_index : 'jupyter-notebook:move-selection-down',
    handler :  function (event) {
        IPython.notebook.move_selection_down();
        return false;
    }}
);

<IPython.core.display.Javascript object>

In [2]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import datetime as dt
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
import lightgbm as lgb
from sklearn.neural_network import MLPClassifier
import featuretools as ft
import gc

In [3]:
df_labels = pd.read_csv('labels_training_set.csv')

In [4]:
df = pd.read_csv('events_up_to_01062018.csv')

/home/amaherok/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df['count'] = 1
df['timestamp'] = pd.to_datetime(df['timestamp'])
df_events = df.loc[df['timestamp'].dt.month == 5]
print(df_events.shape)
df_events = df.groupby(['person', 'event']).agg({'count':'sum'}).reset_index() 
df_events.shape

(1713920, 24)


(232619, 3)

In [6]:
gc.collect()

14

In [7]:
df_events.head()

,person,event,count
0,0008ed71,checkout,3
1,0008ed71,generic listing,1
2,0008ed71,visited site,2
3,00091926,ad campaign hit,15
4,00091926,brand listing,25


In [8]:
df_events = df_events.pivot_table(index='person',columns='event',values='count')
df_events['person'] = df_events.index
df_events = df_events.reset_index(drop=True)

In [9]:
df_events.head()

event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site,person
0,NaN,NaN,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0,0008ed71
1,15.0,25.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,372.0,34.0,00091926
2,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,00091a7a
3,1.0,24.0,6.0,1.0,14.0,NaN,1.0,NaN,NaN,153.0,6.0,000ba417
4,1.0,NaN,1.0,NaN,1.0,NaN,1.0,9.0,NaN,3.0,1.0,000c79fe


In [10]:
ft.primitives.list_primitives().loc[(ft.primitives.list_primitives())['type'] == 'transform']

,name,type,description
19,or,transform,"For two boolean values, determine if one value..."
20,days_since,transform,"For each value of the base feature, compute th..."
21,hours,transform,Transform a Timedelta feature into the number ...
22,minute,transform,Transform a Datetime feature into the minute.
23,days,transform,Transform a Timedelta feature into the number ...
24,numwords,transform,Returns the words in a given string by countin...
25,mod,transform,Creates a transform feature that divides two f...
26,weekday,transform,Transform Datetime feature into the boolean of...
27,haversine,transform,Calculate the approximate haversine distance i...
28,multiply,transform,Creates a transform feature that multplies two...


In [11]:
es = ft.EntitySet(id = 'person')
es = es.entity_from_dataframe(entity_id = 'person_id', dataframe = df_events[['person','checkout','viewed product','conversion','visited site']], index = 'person')

In [12]:
es

Entityset: person
  Entities:
    person_id [Rows: 38829, Columns: 5]
  Relationships:
    No relationships

In [13]:
es

Entityset: person
  Entities:
    person_id [Rows: 38829, Columns: 5]
  Relationships:
    No relationships

In [14]:

%time features, feature_names = ft.dfs(entityset = es, target_entity = 'person_id',trans_primitives = ['subtract','add','diff','percentile'])
features.head()

CPU times: user 6.19 s, sys: 18 ms, total: 6.21 s
Wall time: 6.18 s


event,checkout,viewed product,conversion,visited site,checkout - viewed product,conversion - checkout,conversion - visited site,viewed product - conversion,checkout - conversion,viewed product - checkout,...,PERCENTILE(viewed product - visited site),PERCENTILE(visited site - checkout),PERCENTILE(visited site - viewed product),PERCENTILE(visited site - conversion),PERCENTILE(conversion + visited site),PERCENTILE(checkout + conversion),PERCENTILE(checkout + viewed product),PERCENTILE(viewed product + visited site),PERCENTILE(checkout + visited site),PERCENTILE(conversion + viewed product)
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,3.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.056789,NaN,NaN,NaN,NaN,NaN,NaN,0.616552,NaN
00091926,2.0,372.0,NaN,34.0,-370.0,NaN,NaN,NaN,NaN,370.0,...,0.990956,0.980967,0.009071,NaN,NaN,NaN,0.989534,0.990374,0.971629,NaN
00091a7a,NaN,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.281742,NaN,0.718285,NaN,NaN,NaN,NaN,0.199182,NaN,NaN
000ba417,6.0,153.0,1.0,6.0,-147.0,-5.0,-5.0,152.0,5.0,147.0,...,0.962999,0.270489,0.037029,0.458146,0.410617,0.754937,0.954937,0.952655,0.849331,0.867038
000c79fe,1.0,3.0,NaN,1.0,-2.0,NaN,NaN,NaN,NaN,2.0,...,0.281742,0.270489,0.718285,NaN,NaN,NaN,0.203297,0.199182,0.166450,NaN


In [15]:
df_train = df_labels.merge(features, on='person', how='left')

In [16]:
df_train.head()

,person,label,checkout,viewed product,conversion,visited site,checkout - viewed product,conversion - checkout,conversion - visited site,viewed product - conversion,...,PERCENTILE(viewed product - visited site),PERCENTILE(visited site - checkout),PERCENTILE(visited site - viewed product),PERCENTILE(visited site - conversion),PERCENTILE(conversion + visited site),PERCENTILE(checkout + conversion),PERCENTILE(checkout + viewed product),PERCENTILE(viewed product + visited site),PERCENTILE(checkout + visited site),PERCENTILE(conversion + viewed product)
0,0566e9c1,0,1.0,23.0,1.0,17.0,-22.0,0.0,-16.0,22.0,...,0.456906,0.944899,0.543121,0.756089,0.732561,0.113188,0.660504,0.765441,0.913058,0.411437
1,6ec7ee77,0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,abe7a2fb,0,1.0,31.0,NaN,22.0,-30.0,NaN,NaN,NaN,...,0.537922,0.963080,0.462105,NaN,NaN,NaN,0.728438,0.819474,0.939928,NaN
3,34728364,0,NaN,24.0,NaN,4.0,NaN,NaN,NaN,NaN,...,0.701619,NaN,0.298408,NaN,NaN,NaN,NaN,0.687390,NaN,NaN
4,87ed62de,0,2.0,9.0,NaN,1.0,-7.0,NaN,NaN,NaN,...,0.513363,0.056789,0.486664,NaN,NaN,NaN,0.457695,0.418781,0.406105,NaN


In [17]:
df_train.shape

(19414, 160)

In [18]:
df_train = df_train.fillna(0)

In [19]:

X,y = df_train.iloc[:,2:], df_train.iloc[:,1]

In [20]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

# Modelaje y entrenamiento!

In [21]:
def xgb_classifier(X_train, X_test, y_train, y_test, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    alg = xgb.XGBClassifier(objective ='reg:linear', 
                colsample_bytree = 1, learning_rate = 0.1,
                max_depth = 7,
                subsample = 0.9,
                gamma = 1,
                n_estimators = 50)
    
    print('\nXGBoost Classifier')
    
    alg.fit(X_train, y_train, eval_metric='auc')
    
    pred_proba = alg.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, pred_proba)
    
    print('El puntaje auc es: {}'.format(auc))
    
    return alg

In [22]:
def rf_classifier(X, y):
    RFC = RandomForestClassifier()


    rf_param_grid = {"max_depth": [None],
                  "max_features": [1, 3, 10],
                  "min_samples_split": [2, 3, 10],
                  "min_samples_leaf": [1, 3, 10],
                  "bootstrap": [False],
                  "n_estimators" :[50,65,100],
                  "criterion": ["gini"],
                     "max_features": [0.25]}


    gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=2, scoring="roc_auc", n_jobs= 4, verbose = 1)

    gsRFC.fit(X,y)

    RFC_best = gsRFC.best_estimator_

    # Best score
    print('Random classifier')
    print(gsRFC.best_score_)
    
    return RFC_best

In [23]:
xgb_model = xgb_classifier(X_train, X_test, y_train, y_test)
rf_model = rf_classifier(X,y)


XGBoost Classifier
El puntaje auc es: 0.8232863755225676
Fitting 2 folds for each of 27 candidates, totalling 54 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: 

In [ ]:
zipped = zip(map(lambda x: round(x, 4), rf_model.feature_importances_), X_test.columns)
feature = sorted(zipped, key=lambda x: x[1])

In [ ]:
feat_importance = pd.DataFrame(feature, columns=['importance', 'feature'])
f_m_i = list(feat_importance.sort_values('importance', ascending=False).head(100)['feature'].values)

In [ ]:
feat_importance.head(100)

In [ ]:
#f_m_i =[ x for x in f_m_i if "_y" not in x ]

In [24]:
features['person'] = features.index

In [26]:
features.head()

event,checkout,viewed product,conversion,visited site,checkout - viewed product,conversion - checkout,conversion - visited site,viewed product - conversion,checkout - conversion,viewed product - checkout,...,PERCENTILE(visited site - checkout),PERCENTILE(visited site - viewed product),PERCENTILE(visited site - conversion),PERCENTILE(conversion + visited site),PERCENTILE(checkout + conversion),PERCENTILE(checkout + viewed product),PERCENTILE(viewed product + visited site),PERCENTILE(checkout + visited site),PERCENTILE(conversion + viewed product),person
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,3.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.056789,NaN,NaN,NaN,NaN,NaN,NaN,0.616552,NaN,0008ed71
00091926,2.0,372.0,NaN,34.0,-370.0,NaN,NaN,NaN,NaN,370.0,...,0.980967,0.009071,NaN,NaN,NaN,0.989534,0.990374,0.971629,NaN,00091926
00091a7a,NaN,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.718285,NaN,NaN,NaN,NaN,0.199182,NaN,NaN,00091a7a
000ba417,6.0,153.0,1.0,6.0,-147.0,-5.0,-5.0,152.0,5.0,147.0,...,0.270489,0.037029,0.458146,0.410617,0.754937,0.954937,0.952655,0.849331,0.867038,000ba417
000c79fe,1.0,3.0,NaN,1.0,-2.0,NaN,NaN,NaN,NaN,2.0,...,0.270489,0.718285,NaN,NaN,NaN,0.203297,0.199182,0.166450,NaN,000c79fe


In [27]:
features.to_csv('neww_features.csv' , index=False)